Importiere die requests Library

In [7]:
import requests

Importiere die Pandas Library in der Kurzform.

In [8]:
import pandas as pd

1. Unter Verwendung der [USGS.gov Earthquake API](https://earthquake.usgs.gov) ziehe heraus, wieviele Erdbeben sich im August 2018 ereignet haben?

In [5]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2018-08-01&endtime=2018-08-31')
r.text

'17045'

2. Mache das gleiche für August 2017, 2016, 2015?

In [9]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-08-01&endtime=2017-08-31')
r.text

'12300'

In [10]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2016-08-01&endtime=2016-08-31')
r.text

'10878'

In [11]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2015-08-01&endtime=2015-08-31')
r.text

'9981'

3. Zeige nur die Erdbeben an, die in den letzten 5 Jahren eine Mindeststärke von 5,5 hatten? (Neue Parameter kannst Du mit den Zeichen "&" ergänzen. Lese Dich in die Dokumentation ein, um noch mehr Parameter kennezulernen)

In [14]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2014-09-23&endtime=2019-09-23&minmagnitude=5.5')
r.text

'2233'

4. Zeige eine Liste der 10 stärksten Erdbeben des laufenden Jahres an. (Nicht vergessen, nun musst Du die Methode wechseln. Nicht mehr "count", sondern "query").

In [44]:
r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2019-01-01&endtime=2019-09-23&minmagnitude=5.5')
ebs = r.json()

In [45]:
#Mit For Loop
for key in ebs:
    print(key)

type
metadata
features
bbox


In [46]:
#Oder mit der .keys() Funktion
ebs.keys() 

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [54]:
#Das sind alle Keys der Properties, die uns interessieren
ebs['features'][0]['properties'].keys()

dict_keys(['mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi', 'mmi', 'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids', 'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type', 'title'])

In [65]:
ebs_details = ebs['features']
maglst = []
for eb in ebs_details:
    mag = eb['properties']['mag']
    maglst.append(mag)

In [70]:
maglst.sort()
maglst[-10:]

[6.9, 7, 7.1, 7.1, 7.2, 7.3, 7.3, 7.5, 7.6, 8]

5. Wo haben sich diese Erdbeben ereignet?

In [72]:
ebs_details = ebs['features']
maglst = []
for eb in ebs_details:
    if eb['properties']['mag'] > 6.89:
        mag = eb['properties']['mag']
        place = eb['properties']['place']
        minidict = {'Mag': mag,
                'Place': place}
        maglst.append(minidict)
    else:
        continue

In [74]:
pd.DataFrame(maglst)

,Mag,Place
0,6.9,"106km WSW of Tugu Hilir, Indonesia"
1,7.2,"93km NNE of Laiwui, Indonesia"
2,6.9,"134km WSW of Kota Ternate, Indonesia"
3,7.1,2019 Ridgecrest Earthquake Sequence
4,7.3,"292km NW of Saumlaki, Indonesia"
5,7.3,"116km NE of L'Esperance Rock, New Zealand"
6,8.0,"78km SE of Lagunas, Peru"
7,7.6,"46km SSE of Namatanai, Papua New Guinea"
8,7.1,"33km NW of Bulolo, Papua New Guinea"
9,7.0,"23km NNE of Azangaro, Peru"


6. Ziehe auch die Zeit mit heraus?

In [80]:
ebs_details = ebs['features']
maglst = []
for eb in ebs_details:
    if eb['properties']['mag'] > 6.89:
        mag = eb['properties']['mag']
        place = eb['properties']['place']
        time = eb['properties']['time']
        minidict = {'Mag': mag,
                'Place': place,
                'Time': time}
        maglst.append(minidict)
    else:
        continue

7. Konvertiere die Unix Zeit zu leserlichen Zeit? (Suche nicht zu lange, schau mal an, wies im Pandas aussieht.)

In [82]:
from datetime import datetime
ts = int("1564747406998")
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

ValueError: year 51554 is out of range

In [92]:
#Das Problem sind die letzten drei Ziffer. Die Millisekunden. Sie müssen wir abschneiden.
ts = int("1564747406998"[:-3])
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2019-08-02 12:03:26


In [95]:
ebs_details = ebs['features']
maglst = []
for eb in ebs_details:
    if eb['properties']['mag'] > 6.89:
        mag = eb['properties']['mag']
        place = eb['properties']['place']
        time = str(eb['properties']['time'])[:-3]
        time = datetime.utcfromtimestamp(int(time)).strftime('%Y-%m-%d %H:%M:%S')
        minidict = {'Mag': mag,
                'Place': place,
                'Time': time}
        maglst.append(minidict)
    else:
        continue

In [101]:
#Sortieren wir die Tabelle (aber das werdet ihr am Mittwoch und Donnerstag genauer anschauen)
pd.DataFrame(maglst).sort_values(by='Time', ascending=False)

,Mag,Place,Time
0,6.9,"106km WSW of Tugu Hilir, Indonesia",2019-08-02 12:03:26
1,7.2,"93km NNE of Laiwui, Indonesia",2019-07-14 09:10:51
2,6.9,"134km WSW of Kota Ternate, Indonesia",2019-07-07 15:08:40
3,7.1,2019 Ridgecrest Earthquake Sequence,2019-07-06 03:19:53
4,7.3,"292km NW of Saumlaki, Indonesia",2019-06-24 02:53:39
5,7.3,"116km NE of L'Esperance Rock, New Zealand",2019-06-15 22:55:04
6,8.0,"78km SE of Lagunas, Peru",2019-05-26 07:41:15
7,7.6,"46km SSE of Namatanai, Papua New Guinea",2019-05-14 12:58:25
8,7.1,"33km NW of Bulolo, Papua New Guinea",2019-05-06 21:19:37
9,7.0,"23km NNE of Azangaro, Peru",2019-03-01 08:50:42
